In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("parkinson_10%.csv")

In [3]:
df

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,UPDRS,class
0,1.49,0.0,NaN,0.79,2.70,8.33,0.78,4.52,4.61,6.80,...,187.58,160.0,NaN,0.01,0.0,0.00,0.0,0.00,23.0,1
1,0.73,0.0,0.35,0.38,NaN,5.86,0.64,2.06,3.18,7.19,...,234.51,170.0,169.0,0.01,0.0,2.25,0.0,0.00,23.0,1
2,1.22,0.0,0.73,0.67,NaN,8.72,0.88,4.35,5.17,NaN,...,211.44,1431.0,NaN,0.01,0.0,10.66,1.0,0.18,23.0,1
3,2.50,0.0,1.16,NaN,3.47,13.51,NaN,5.26,8.77,16.78,...,220.23,94.0,92.0,0.00,0.0,0.00,0.0,0.00,23.0,1
4,NaN,0.0,1.72,NaN,5.15,9.11,1.04,3.10,4.93,12.82,...,NaN,117.0,114.0,0.00,0.0,18.18,NaN,13.32,23.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,5.13,0.0,2.72,2.51,8.16,9.85,0.97,4.59,6.81,8.30,...,158.59,63.0,60.0,0.01,0.0,24.19,2.0,15.11,1.0,0
1036,1.76,0.0,NaN,0.69,2.43,NaN,0.74,2.72,3.93,NaN,...,143.80,47.0,44.0,0.01,0.0,NaN,NaN,25.79,1.0,0
1037,2.91,0.0,1.14,1.27,3.42,15.49,1.46,6.43,10.14,14.24,...,149.64,79.0,77.0,0.01,0.0,20.00,1.0,4.57,1.0,0
1038,3.79,0.0,1.79,1.48,5.36,15.70,1.37,6.37,11.09,NaN,...,451.37,75.0,70.0,0.01,0.0,16.67,3.0,19.58,1.0,0


In [4]:
df.dtypes

Jitter (local)                         float64
Jitter (local, absolute)               float64
Jitter (rap)                           float64
Jitter (ppq5)                          float64
Jitter (ddp)                           float64
Shimmer (local)                        float64
Shimmer (local, dB)                    float64
Shimmer (apq3)                         float64
Shimmer (apq5)                         float64
Shimmer (apq11)                        float64
Shimmer (dda)                          float64
AC                                     float64
NTH                                    float64
HTN                                    float64
Median pitch                           float64
Mean pitch                             float64
Standard deviation                     float64
Minimum pitch                          float64
Maximum pitch                          float64
Number of pulses                       float64
Number of periods                      float64
Mean period  

In [5]:
df.isnull().sum()

Jitter (local)                         110
Jitter (local, absolute)               119
Jitter (rap)                           110
Jitter (ppq5)                          112
Jitter (ddp)                           112
Shimmer (local)                        106
Shimmer (local, dB)                    101
Shimmer (apq3)                         100
Shimmer (apq5)                          99
Shimmer (apq11)                         94
Shimmer (dda)                          106
AC                                     110
NTH                                    110
HTN                                    111
Median pitch                           119
Mean pitch                             112
Standard deviation                     103
Minimum pitch                          125
Maximum pitch                          119
Number of pulses                       102
Number of periods                      111
Mean period                            113
Standard deviation of period           100
Fraction of

In [6]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 29120
nilai hilang : 2912
persentase : 10.0


In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Jitter (local)                         10.576923
Jitter (local, absolute)               11.442308
Jitter (rap)                           10.576923
Jitter (ppq5)                          10.769231
Jitter (ddp)                           10.769231
Shimmer (local)                        10.192308
Shimmer (local, dB)                     9.711538
Shimmer (apq3)                          9.615385
Shimmer (apq5)                          9.519231
Shimmer (apq11)                         9.038462
Shimmer (dda)                          10.192308
AC                                     10.576923
NTH                                    10.576923
HTN                                    10.673077
Median pitch                           11.442308
Mean pitch                             10.769231
Standard deviation                      9.903846
Minimum pitch                          12.019231
Maximum pitch                          11.442308
Number of pulses                        9.807692
Number of periods   

# Imputasi

In [8]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",euc_dist[col])

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [9]:
from collections import Counter
def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count == 1 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None
    
def Final(df,col,k):
    terdekat = []
    mean = 0
    std = 0
    diffsquared = 0
    sum_diffsquared = 0
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        mean += y  
        euc_val.append(y)
    mean /= len(terdekat)
    
    print(euc_val)
    
    for x,y in terdekat:
        diffsquared = (y-mean)**2
        sum_diffsquared = diffsquared + sum_diffsquared
    std_dev = ((sum_diffsquared)/len(terdekat))**(1/2)
    print("mean : ",mean)
    print("standar deviasi : ",std_dev)
    
    euc_val.sort()
    fuzzy = hitung(euc_val,7)
    
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,gaussian(y,mean,std_dev)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    
    if tertimbang == 0:
        tertimbang = dummy.iloc[index-1, col]
        
    dummy.iloc[index,col] = tertimbang
    print(" ")

In [10]:
def gaussian(val, mean, std_dev):
    gaussian_mf = 2.718**((-1 * (val-mean)**2) / (2 * std_dev**2))
    return gaussian_mf

In [11]:
data = df.iloc[:, :-1]
df2 = data.copy()
dummy = df2.fillna(0)
dummy

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Minimum pitch,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,UPDRS
0,1.49,0.0,0.00,0.79,2.70,8.33,0.78,4.52,4.61,6.80,...,142.23,187.58,160.0,0.0,0.01,0.0,0.00,0.0,0.00,23.0
1,0.73,0.0,0.35,0.38,0.00,5.86,0.64,2.06,3.18,7.19,...,159.52,234.51,170.0,169.0,0.01,0.0,2.25,0.0,0.00,23.0
2,1.22,0.0,0.73,0.67,0.00,8.72,0.88,4.35,5.17,0.00,...,0.00,211.44,1431.0,0.0,0.01,0.0,10.66,1.0,0.18,23.0
3,2.50,0.0,1.16,0.00,3.47,13.51,0.00,5.26,8.77,16.78,...,182.71,220.23,94.0,92.0,0.00,0.0,0.00,0.0,0.00,23.0
4,0.00,0.0,1.72,0.00,5.15,9.11,1.04,3.10,4.93,12.82,...,182.82,0.00,117.0,114.0,0.00,0.0,18.18,0.0,13.32,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,5.13,0.0,2.72,2.51,8.16,9.85,0.97,4.59,6.81,8.30,...,96.15,158.59,63.0,60.0,0.01,0.0,24.19,2.0,15.11,1.0
1036,1.76,0.0,0.00,0.69,2.43,0.00,0.74,2.72,3.93,0.00,...,121.49,143.80,47.0,44.0,0.01,0.0,0.00,0.0,25.79,1.0
1037,2.91,0.0,1.14,1.27,3.42,15.49,1.46,6.43,10.14,14.24,...,0.00,149.64,79.0,77.0,0.01,0.0,20.00,1.0,4.57,1.0
1038,3.79,0.0,1.79,1.48,5.36,15.70,1.37,6.37,11.09,0.00,...,90.05,451.37,75.0,70.0,0.01,0.0,16.67,3.0,19.58,1.0


In [12]:
count_zeros = np.count_nonzero(dummy == 0)

print("Jumlah angka 0 dalam dataset:", count_zeros)

Jumlah angka 0 dalam dataset: 5922


In [13]:
euc_dist = {}

In [14]:
for i in range(dummy.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,7)

Kolom : 1
[0.0, 2.68, 2.16, 4.34, 2.57, 0.89, 2.33]
mean :  2.138571428571429
standar deviasi :  1.281465997223811
fuzzy :  [(0.0, 0.24848277794077148), (2.68, 0.9146197263373641), (2.16, 0.9998602129691059), (4.34, 0.22867929641273207), (2.57, 0.9449088709219091), (0.89, 0.6221271635396667), (2.33, 0.9889055957920617)]
fuzzy :  None
pembilang :  10.889606085043775
penyebut :  4.947583643913611
rata-rata tertimbang :  2.2009948428946493
 
[2.11, 2.29, 1.29, 2.5, 2.72, 3.05, 1.53]
mean :  2.212857142857143
standar deviasi :  0.5824753651618308
fuzzy :  [(2.11, 0.984531154903971), (2.29, 0.9912691110236929), (1.29, 0.2850799029484167), (2.5, 0.885593628815197), (2.72, 0.6845508314380259), (3.05, 0.3560487250291361), (1.53, 0.5030251006722652)]
fuzzy :  None
pembilang :  10.646659424811947
penyebut :  4.6900984548307045
rata-rata tertimbang :  2.2700289828342743
 
[0.0, 1.63, 2.83, 1.16, 0.41, 1.24, 2.36]
mean :  1.375714285714286
standar deviasi :  0.9287911827878697
fuzzy :  [(0.0, 0.33

[3.44, 0.0, 2.69, 4.24, 1.27, 2.31, 5.94]
mean :  2.8414285714285716
standar deviasi :  1.8033257258380662
fuzzy :  [(3.44, 0.9464076736767857), (0.0, 0.28903027976248324), (2.69, 0.9964809374293221), (4.24, 0.7402934265447474), (1.27, 0.6841112666984567), (2.31, 0.9575113921154332), (5.94, 0.22853974555221732)]
fuzzy :  None
pembilang :  13.51321896075661
penyebut :  4.8423747217794455
rata-rata tertimbang :  2.7906181857381864
 
[3.44, 2.7906181857381864, 2.69, 4.24, 2.02, 2.31, 3.68]
mean :  3.0243740265340264
standar deviasi :  0.7334733505682981
fuzzy :  [(3.44, 0.8516905572519016), (2.7906181857381864, 0.9504890284187899), (2.69, 0.9013141702145282), (4.24, 0.25327714418558256), (2.02, 0.3916259453308298), (2.31, 0.6223514641019166), (3.68, 0.6706850176649173)]
fuzzy :  None
pembilang :  13.777534850871186
penyebut :  4.641433327168466
rata-rata tertimbang :  2.968379351745693
 
[2.42, 4.82, 2.74, 3.2, 2.23, 1.93, 2.43]
mean :  2.8242857142857143
standar deviasi :  0.894664456539

[0.0, 1.84, 2.47, 2.33, 1.8, 3.09, 1.29]
mean :  1.8314285714285714
standar deviasi :  0.917191809997522
fuzzy :  [(0.0, 0.13623760773766147), (1.84, 0.9999563382060396), (2.47, 0.7847899611239734), (2.33, 0.8626667281483417), (1.8, 0.9994131508847974), (3.09, 0.3900930253677519), (1.29, 0.8401157709853504)]
fuzzy :  None
pembilang :  9.876444807411055
penyebut :  5.013272582453915
rata-rata tertimbang :  1.9700594063003645
 
[0.0, 4.46, 3.3, 2.32, 2.57, 3.35, 4.32]
mean :  2.902857142857143
standar deviasi :  1.397331275638512
fuzzy :  [(0.0, 0.11559843935246492), (4.46, 0.5374901118416916), (3.3, 0.9604196387771397), (2.32, 0.9166897274773754), (2.57, 0.9720297519915221), (3.35, 0.9500943630211103), (4.32, 0.5979636313931155)]
fuzzy :  None
pembilang :  15.957446340883205
penyebut :  5.050285663854419
rata-rata tertimbang :  3.1597116288080134
 
[0.0, 0.0, 3.04, 2.070550182748898, 2.69, 1.72, 4.33]
mean :  1.9786500261069855
standar deviasi :  1.4671053413629203
fuzzy :  [(0.0, 0.402

[0.0, 3.41, 0.67, 2.4632107779676327, 1.65, 1.7, 1.35]
mean :  1.6061729682810901
standar deviasi :  1.0365372371808583
fuzzy :  [(0.0, 0.30106227965325577), (3.41, 0.22001348180575458), (0.67, 0.6650993172990529), (2.4632107779676327, 0.7104997602838908), (1.65, 0.9991066015383447), (1.7, 0.9959118965811201), (1.35, 0.9699248587798796)]
fuzzy :  None
pembilang :  7.596947858901698
penyebut :  4.861618195941299
rata-rata tertimbang :  1.5626376964863218
 
[0.0, 1.202881105946006, 2.24, 5.68, 2.4632107779676327, 4.9, 2.33]
mean :  2.6880131262733764
standar deviasi :  1.8382961180168982
fuzzy :  [(0.0, 0.34336936328240675), (1.202881105946006, 0.7215848386889108), (2.24, 0.9707420836415311), (5.68, 0.26596717018358185), (2.4632107779676327, 0.9925514242384201), (4.9, 0.48487334084609846), (2.33, 0.9812163140247915)]
fuzzy :  None
pembilang :  11.66011331049061
penyebut :  4.760304534905741
rata-rata tertimbang :  2.449446926134841
 
[0.0, 3.99, 1.64, 2.33, 3.51, 2.36, 1.4]
mean :  2.175

[0.0, 3.66, 3.64, 1.37, 8.78, 1.63, 6.93]
mean :  3.7157142857142857
standar deviasi :  2.919064709030994
fuzzy :  [(0.0, 0.4448272565240724), (3.66, 0.9998178911238367), (3.64, 0.9996637052007139), (1.37, 0.7240894763074397), (8.78, 0.22206543798621894), (1.63, 0.7747312582387281), (6.93, 0.5454267233166632)]
fuzzy :  None
pembilang :  15.282465640017637
penyebut :  4.7106217486976725
rata-rata tertimbang :  3.2442565876240694
 
[0.0, 0.4, 2.73, 1.77, 0.41, 2.8, 4.16]
mean :  1.752857142857143
standar deviasi :  1.4418214104360625
fuzzy :  [(0.0, 0.4776313819921766), (0.4, 0.6439364268491323), (2.73, 0.7948320346357091), (1.77, 0.9999293270699829), (0.41, 0.6481246268358654), (2.8, 0.7682022071603504), (4.16, 0.24820602917300003)]
fuzzy :  None
pembilang :  7.646575292620375
penyebut :  4.580862033716216
rata-rata tertimbang :  1.6692437441555306
 
[0.0, 3.33, 1.5, 1.05, 1.26, 1.29, 1.81]
mean :  1.4628571428571429
standar deviasi :  0.9250129618507174
fuzzy :  [(0.0, 0.28640324271104

...

[1.0, 46.0, 1.0, 8.0, 8.0, 1.0, 8.0]
mean :  10.428571428571429
standar deviasi :  14.879104647190328
fuzzy :  [(1.0, 0.8181140561586444), (46.0, 0.05741673843535842), (1.0, 0.8181140561586444), (8.0, 0.9867692594061167), (8.0, 0.9867692594061167), (1.0, 0.8181140561586444), (8.0, 0.9867692594061167)]
fuzzy :  None
pembilang :  28.777974362249225
penyebut :  5.472066685129642
rata-rata tertimbang :  5.259068651420762
 
[16.0, 1.0, 32.0, 1.0, 11.0, 1.0, 5.0]
mean :  9.571428571428571
standar deviasi :  10.608044694706841
fuzzy :  [(16.0, 0.8322649286912177), (1.0, 0.7215107959561647), (32.0, 0.10700314895665251), (1.0, 0.7215107959561647), (11.0, 0.9909740931158354), (1.0, 0.7215107959561647), (5.0, 0.9113348150473141)]
fuzzy :  None
pembilang :  34.362261113051616
penyebut :  5.006109373679514
rata-rata tertimbang :  6.864065194763252
 
[1.0, 5.0, 1.0, 11.0, 46.0, 1.0, 46.0]
mean :  15.857142857142858
standar deviasi :  19.349629538192872
fuzzy :  [(1.0, 0.7447195388366971), (5.0, 0.85

[0.0, 1.0, 1.0, 26.0, 0.0, 24.0, 24.0]
mean :  10.857142857142858
standar deviasi :  11.981277912149574
fuzzy :  [(0.0, 0.6632955402336577), (1.0, 0.7129153797647865), (1.0, 0.7129153797647865), (26.0, 0.4499543007345805), (0.0, 0.6632955402336577), (24.0, 0.5479414990824487), (24.0, 0.5479414990824487)]
fuzzy :  None
pembilang :  39.4258345345862
penyebut :  4.298259138896366
rata-rata tertimbang :  9.172512233571217
 
[0.0, 5.0, 1.0, 20.0, 14.777272946773374, 1.0, 5.0]
mean :  6.682467563824767
standar deviasi :  7.14922759616579
fuzzy :  [(0.0, 0.6461020582043298), (5.0, 0.9726912696758627), (1.0, 0.7291689361265138), (20.0, 0.17643207324268953), (14.777272946773374, 0.5267953119092785), (1.0, 0.7291689361265138), (5.0, 0.9726912696758627)]
fuzzy :  None
pembilang :  22.49849014502947
penyebut :  4.75304985496105
rata-rata tertimbang :  4.73348498996837
 
[0.0, 26.0, 23.0, 5.0, 5.0, 26.0, 5.0]
mean :  12.857142857142858
standar deviasi :  10.682810911908524
fuzzy :  [(0.0, 0.4847248

[0.0, 1.0, 11.0, 1.0, 4.673060665324237, 40.0, 8.0]
mean :  9.381865809332036
standar deviasi :  13.053196945749162
fuzzy :  [(0.0, 0.7723886519633302), (1.0, 0.8137151590261733), (11.0, 0.9923466244149874), (1.0, 0.8137151590261733), (4.673060665324237, 0.9370114416620703), (40.0, 0.06388093924048267), (8.0, 0.9944126378722146)]
fuzzy :  None
pembilang :  27.432493170204012
penyebut :  5.387470613205432
rata-rata tertimbang :  5.091905857075711
 
[0.0, 23.0, 1.0, 1.0, 23.0, 1.0, 23.0]
mean :  10.285714285714286
standar deviasi :  11.01575865637543
fuzzy :  [(0.0, 0.646696546069956), (23.0, 0.5137549008557347), (1.0, 0.7010023609029724), (1.0, 0.7010023609029724), (23.0, 0.5137549008557347), (1.0, 0.7010023609029724), (23.0, 0.5137549008557347)]
fuzzy :  None
pembilang :  37.552095241754614
penyebut :  4.290968331346077
rata-rata tertimbang :  8.751426797404147
 
[0.0, 1.0, 1.0, 1.0, 55.0, 55.0, 12.0]
mean :  17.857142857142858
standar deviasi :  23.79933115163756
fuzzy :  [(0.0, 0.754

[0.0, 1.0, 40.0, 8.0, 8.0, 40.0, 1.0]
mean :  14.0
standar deviasi :  16.724661004465403
fuzzy :  [(0.0, 0.7044617935672149), (1.0, 0.7392917460352793), (40.0, 0.29871940106499), (8.0, 0.9376817186142132), (8.0, 0.9376817186142132), (40.0, 0.29871940106499), (1.0, 0.7392917460352793)]
fuzzy :  None
pembilang :  40.37904307509717
penyebut :  4.6558475249961795
rata-rata tertimbang :  8.672758903359986
 
[0.0, 26.0, 26.0, 0.0, 5.0, 1.0, 1.0]
mean :  8.428571428571429
standar deviasi :  11.223153909372774
fuzzy :  [(0.0, 0.7542962278365443), (26.0, 0.29361420341311273), (26.0, 0.29361420341311273), (0.0, 0.7542962278365443), (5.0, 0.9544142209243061), (1.0, 0.803296752199939), (1.0, 0.803296752199939)]
fuzzy :  None
pembilang :  21.646603186503267
penyebut :  4.656828587823498
rata-rata tertimbang :  4.6483573054640654
 
[0.0, 1.0, 26.0, 1.0, 1.0, 1.0, 9.193507023505218]
mean :  5.599072431929317
standar deviasi :  8.824262710644554
fuzzy :  [(0.0, 0.8176835762751536), (1.0, 0.87301480442

[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 23.0]
mean :  4.0
standar deviasi :  7.764387566686396
fuzzy :  [(0.0, 0.8757385751746958), (1.0, 0.9280804944047584), (1.0, 0.9280804944047584), (1.0, 0.9280804944047584), (1.0, 0.9280804944047584), (1.0, 0.9280804944047584), (23.0, 0.05009844091613034)]
fuzzy :  None
pembilang :  5.79266661309479
penyebut :  5.566239488114618
rata-rata tertimbang :  1.0406786530589731
 
[0.0, 23.0, 1.0, 31.0, 23.0, 46.0, 40.0]
mean :  23.428571428571427
standar deviasi :  16.447814816713432
fuzzy :  [(0.0, 0.362627616469694), (23.0, 0.9996606238789615), (1.0, 0.39469832689740425), (31.0, 0.8994776831634455), (23.0, 0.9996606238789615), (46.0, 0.3900369832455493), (40.0, 0.6020040359452843)]
fuzzy :  None
pembilang :  116.28475787050309
penyebut :  4.6481658934793
rata-rata tertimbang :  25.01734243901098
 
[0.0, 20.0, 1.0, 26.0, 1.0, 20.0, 1.0]
mean :  9.857142857142858
standar deviasi :  10.682810911908524
fuzzy :  [(0.0, 0.6533428254602628), (20.0, 0.6371905979391251)

In [15]:
dummy

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Minimum pitch,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,UPDRS
0,1.490000,0.0,0.881623,0.790000,2.700000,8.330000,0.780000,4.52,4.610000,6.800000,...,142.230000,187.580000,160.0,171.121645,0.010000,0.0,4.749621,0.017882,0.193931,23.0
1,0.730000,0.0,0.350000,0.380000,1.568146,5.860000,0.640000,2.06,3.180000,7.190000,...,159.520000,234.510000,170.0,169.000000,0.010000,0.0,2.250000,0.187787,10.761204,23.0
2,1.220000,0.0,0.730000,0.670000,0.761105,8.720000,0.880000,4.35,5.170000,5.009994,...,135.644675,211.440000,1431.0,1080.115942,0.010000,0.0,10.660000,1.000000,0.180000,23.0
3,2.500000,0.0,1.160000,1.219860,3.470000,13.510000,0.809965,5.26,8.770000,16.780000,...,182.710000,220.230000,94.0,92.000000,0.001228,0.0,16.187962,1.000000,0.231394,23.0
4,2.200995,0.0,1.720000,0.994616,5.150000,9.110000,1.040000,3.10,4.930000,12.820000,...,182.820000,236.120448,117.0,114.000000,0.001228,0.0,18.180000,0.008941,13.320000,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,5.130000,0.0,2.720000,2.510000,8.160000,9.850000,0.970000,4.59,6.810000,8.300000,...,96.150000,158.590000,63.0,60.000000,0.010000,0.0,24.190000,2.000000,15.110000,1.0
1036,1.760000,0.0,1.057102,0.690000,2.430000,8.225291,0.740000,2.72,3.930000,7.236441,...,121.490000,143.800000,47.0,44.000000,0.010000,0.0,17.859730,1.380447,25.790000,1.0
1037,2.910000,0.0,1.140000,1.270000,3.420000,15.490000,1.460000,6.43,10.140000,14.240000,...,102.670659,149.640000,79.0,77.000000,0.010000,0.0,20.000000,1.000000,4.570000,1.0
1038,3.790000,0.0,1.790000,1.480000,5.360000,15.700000,1.370000,6.37,11.090000,9.967414,...,90.050000,451.370000,75.0,70.000000,0.010000,0.0,16.670000,3.000000,19.580000,1.0


In [16]:
dummy.describe()

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Minimum pitch,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,UPDRS
count,1040.000000,1040.0,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,...,1040.000000,1040.000000,1040.000000,1040.000000,1.040000e+03,1040.00000,1040.000000,1040.000000,1040.000000,1040.000000
mean,2.631758,0.0,1.212384,1.320004,3.637555,12.752605,1.175480,5.588404,7.849976,12.007658,...,133.411580,231.961134,108.442335,104.292548,7.871834e-03,0.00001,27.959789,1.243321,13.835116,12.570069
std,1.711631,0.0,0.910726,1.105303,2.809989,5.312529,0.406068,2.874158,4.724016,5.838261,...,45.822682,119.567492,149.760655,143.329153,3.844608e-03,0.00031,19.579390,1.496839,13.426576,15.381783
min,0.190000,0.0,0.060000,0.080000,0.190000,1.190000,0.100000,0.500000,0.710000,0.520000,...,67.960000,85.540000,6.000000,4.000000,1.057619e-09,0.00000,0.160000,0.000007,0.000007,0.880402
25%,1.530000,0.0,0.630000,0.670000,1.867500,9.385000,0.936099,3.699986,5.172175,8.159347,...,101.192927,143.027500,42.000000,40.000000,8.772004e-03,0.00000,10.852913,0.259144,3.229877,1.000000
50%,2.350000,0.0,1.010000,1.100000,3.035000,12.080000,1.160000,5.050000,6.925000,11.260000,...,126.430000,195.420000,64.000000,61.000000,1.000000e-02,0.00000,25.930000,1.000000,9.410942,5.000000
75%,3.332500,0.0,1.560000,1.622500,4.672500,14.960000,1.380000,6.742500,9.220000,14.810000,...,157.037500,260.812500,111.000000,108.000000,1.000000e-02,0.00000,41.858459,1.123015,21.602500,23.000000
max,14.380000,0.0,8.020000,13.540000,24.050000,41.140000,2.720000,25.820000,72.860000,44.760000,...,452.080000,526.410000,1490.000000,1483.000000,1.000000e-02,0.01000,88.160000,12.000000,69.120000,55.000000


# Klasifikasi

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [18]:
x = dummy.values
y = df.iloc[:, -1].values

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=45)

In [20]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [21]:
classifier = DecisionTreeClassifier()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[129   6]
 [  7 118]]
0.95
